In [12]:
import pandas as pd
import numpy as np

In [13]:
train = pd.read_csv("../../data/raw/train.csv")
test = pd.read_csv("../../data/raw/test.csv")

In [14]:
train.columns

Index(['id', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

In [15]:
train = train.drop(['origin', 'cylinders', 'car name'], axis=1)
test_x = test.drop(['origin', 'cylinders', 'car name'], axis=1)

In [17]:
train_y = train['mpg']
train_x = train.drop(['mpg'], axis=1)

In [18]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [25]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.preprocessing import StandardScaler

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# ニューラルネットモデルの構築
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size = 128
epochs = 100
history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y))

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = mean_squared_error(va_y, va_pred)
print(f'mean_absolute_error: {score:.4f}')

# 予測
pred = model.predict(test_x)

# -----------------------------------
# アーリーストッピング
# -----------------------------------
from keras.callbacks import EarlyStopping

# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
epochs = 100
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])
pred = model.predict(test_x)

Train on 149 samples, validate on 50 samples
Epoch 1/100
149/149 [==============================] - 1s 8ms/step - loss: 24.1829 - mean_absolute_error: 24.1829 - val_loss: 24.6371 - val_mean_absolute_error: 24.6371
Epoch 2/100
149/149 [==============================] - 0s 120us/step - loss: 23.9210 - mean_absolute_error: 23.9210 - val_loss: 24.3726 - val_mean_absolute_error: 24.3726
Epoch 3/100
149/149 [==============================] - 0s 123us/step - loss: 23.6642 - mean_absolute_error: 23.6642 - val_loss: 24.1011 - val_mean_absolute_error: 24.1011
Epoch 4/100
149/149 [==============================] - 0s 104us/step - loss: 23.3904 - mean_absolute_error: 23.3904 - val_loss: 23.8054 - val_mean_absolute_error: 23.8054
Epoch 5/100
149/149 [==============================] - 0s 114us/step - loss: 23.1016 - mean_absolute_error: 23.1016 - val_loss: 23.4758 - val_mean_absolute_error: 23.4758
Epoch 6/100
149/149 [==============================] - 0s 99us/step - loss: 22.7715 - mean_absolute_er

149/149 [==============================] - 0s 95us/step - loss: 3.2634 - mean_absolute_error: 3.2634 - val_loss: 3.1922 - val_mean_absolute_error: 3.1922
Epoch 50/100
149/149 [==============================] - 0s 95us/step - loss: 3.2438 - mean_absolute_error: 3.2438 - val_loss: 3.1887 - val_mean_absolute_error: 3.1887
Epoch 51/100
149/149 [==============================] - 0s 97us/step - loss: 3.3117 - mean_absolute_error: 3.3117 - val_loss: 3.1566 - val_mean_absolute_error: 3.1566
Epoch 52/100
149/149 [==============================] - 0s 87us/step - loss: 3.3444 - mean_absolute_error: 3.3444 - val_loss: 3.1074 - val_mean_absolute_error: 3.1074
Epoch 53/100
149/149 [==============================] - 0s 80us/step - loss: 3.4292 - mean_absolute_error: 3.4292 - val_loss: 3.0639 - val_mean_absolute_error: 3.0639
Epoch 54/100
149/149 [==============================] - 0s 80us/step - loss: 3.1029 - mean_absolute_error: 3.1029 - val_loss: 3.0490 - val_mean_absolute_error: 3.0490
Epoch 55/10

149/149 [==============================] - 0s 87us/step - loss: 2.8652 - mean_absolute_error: 2.8652 - val_loss: 2.5357 - val_mean_absolute_error: 2.5357
Epoch 99/100
149/149 [==============================] - 0s 89us/step - loss: 2.6289 - mean_absolute_error: 2.6289 - val_loss: 2.5108 - val_mean_absolute_error: 2.5108
Epoch 100/100
149/149 [==============================] - 0s 93us/step - loss: 2.7133 - mean_absolute_error: 2.7133 - val_loss: 2.4936 - val_mean_absolute_error: 2.4936
mean_absolute_error: 11.6048
Train on 149 samples, validate on 50 samples
Epoch 1/100
149/149 [==============================] - 0s 81us/step - loss: 2.8205 - mean_absolute_error: 2.8205 - val_loss: 2.5084 - val_mean_absolute_error: 2.5084
Epoch 2/100
149/149 [==============================] - 0s 92us/step - loss: 2.7328 - mean_absolute_error: 2.7328 - val_loss: 2.5023 - val_mean_absolute_error: 2.5023
Epoch 3/100
149/149 [==============================] - 0s 86us/step - loss: 2.9502 - mean_absolute_error:

Epoch 47/100
149/149 [==============================] - 0s 88us/step - loss: 2.3794 - mean_absolute_error: 2.3794 - val_loss: 2.3704 - val_mean_absolute_error: 2.3704
Epoch 48/100
149/149 [==============================] - 0s 93us/step - loss: 2.3265 - mean_absolute_error: 2.3265 - val_loss: 2.3594 - val_mean_absolute_error: 2.3594
Epoch 49/100
149/149 [==============================] - 0s 90us/step - loss: 2.3543 - mean_absolute_error: 2.3543 - val_loss: 2.2566 - val_mean_absolute_error: 2.2566
